In [1]:
using IJulia
installkernel("julia_ITensors","--sysimage=~/.julia/sysimages/sys_itensors.so")

[ Info: Installing julia_ITensors kernelspec in /Users/mschuylerm/Library/Jupyter/kernels/julia_itensors-1.9


"/Users/mschuylerm/Library/Jupyter/kernels/julia_itensors-1.9"

In [2]:
using ITensors
using ITensors.HDF5
using DataFrames

# Lattice, DMRG function

In [3]:
function spinchain_open(L)
    
    interactions = DataFrame(["$spin"=>[] for spin in 1:L-1])
    
    for spin in 1:L-1
        neighbor = spin+1
        push!(interactions[!,"$spin"],[spin,neighbor])
    end

    return interactions
end

spinchain_open (generic function with 1 method)

In [11]:
function dmrg_1D_ising(L::Int, J, maxbd::Int, bc::String, save::Bool)
    
    if bc == "open"
        interactions = spinchain_open(L)
    elseif bc == "peri"
        interactions = spinchain_peri(L)
    end
    
    sites = siteinds("S=1/2",L)
    println("\nRunning DMRG for a length $L spin chain\n")
    
    # Build Hamiltonian
    os = OpSum()
    for k=1:ncol(interactions)
        i = interactions[!,k][1][1]
        j = interactions[!,k][1][2]
        os += 4*J,"Sz",i,"Sz",j
    end
    H = MPO(os,sites)

    psi0 = randomMPS(sites, 40)

    sweeps = Sweeps(10)
    setmaxdim!(sweeps, 10,20,100,100,maxbd)
    setcutoff!(sweeps, 1E-10)

    energy, psi = dmrg(H, psi0, sweeps, outputlevel=1)
    energy_per_spin = energy / L
    
    energy_est = inner(psi',H,psi)
    energy_per_spin_est = energy_est/L
    
    println("\nA sample from the optimized MPS looks like:\n",sample(psi))
    
    if save   
        f = h5open("../out/ising_fm_1d_L$(L)/sanity_check/init.hdf5","w") 
        write(f,"psi",psi)
        close(f)
    end
    
    for i =1:L
        println("M",i)
        println(psi[i])
    end
    
    return energy_per_spin, energy_per_spin_est, psi
end

dmrg_1D_ising (generic function with 1 method)

In [123]:
function dmrg_1D_tfim(L::Int, J, h, maxbd::Int, bc::String, save::Bool)
    
    if bc == "open"
        interactions = spinchain_open(L)
    elseif bc == "peri"
        interactions = spinchain_peri(L)
    end
    
    sites = siteinds("S=1/2",L)
    println("\nRunning DMRG for a length $L spin chain\n")
    
    # Build Hamiltonian
    os = OpSum()
    for k=1:ncol(interactions)
        i = interactions[!,k][1][1]
        j = interactions[!,k][1][2]
        os += 4*J,"Sz",i,"Sz",j
    end
    for k=1:L
        os += 2*h,"Sx",k
    end
    H = MPO(os,sites)

    psi0 = randomMPS(sites, 40)

    sweeps = Sweeps(10)
    setmaxdim!(sweeps, 10,20,100,100,maxbd)
    setmindim!(sweeps,maxbd)
    setcutoff!(sweeps, 1E-10)

    energy, psi = dmrg(H, psi0, sweeps, outputlevel=1)
    energy_per_spin = energy / L
    
    energy_est = inner(psi',H,psi)
    energy_per_spin_est = energy_est/L
    
    if save   
        f = h5open("tfim_J$(Int(J))_h$(Int(h))_1d_L$(L).h5","w") 
        write(f,"psi",psi)
        close(f)
    end
    
    println("")
    @show psi
    
    return energy_per_spin,energy_per_spin_est,psi
end

dmrg_1D_tfim (generic function with 1 method)

# Try growing algo

Step 1: Obtain wavefunction for a two-site lattice and a four-site lattice. Set n = 1. 

In [132]:
_,_,psi0 = dmrg_1D_ising(2,-1,2,"open",false)
_,_,psi1 = dmrg_1D_ising(4,-1,2,"open",false)


Running DMRG for a length 2 spin chain

After sweep 1 energy=-1.0  maxlinkdim=2 maxerr=0.00E+00 time=0.001
After sweep 2 energy=-1.0  maxlinkdim=2 maxerr=0.00E+00 time=0.001
After sweep 3 energy=-1.0  maxlinkdim=2 maxerr=0.00E+00 time=0.001
After sweep 4 energy=-1.0  maxlinkdim=2 maxerr=0.00E+00 time=0.001
After sweep 5 energy=-1.0  maxlinkdim=2 maxerr=0.00E+00 time=0.001
After sweep 6 energy=-1.0  maxlinkdim=2 maxerr=0.00E+00 time=0.001
After sweep 7 energy=-1.0  maxlinkdim=2 maxerr=0.00E+00 time=0.000
After sweep 8 energy=-1.0  maxlinkdim=2 maxerr=0.00E+00 time=0.000
After sweep 9 energy=-1.0  maxlinkdim=2 maxerr=0.00E+00 time=0.001
After sweep 10 energy=-1.0  maxlinkdim=2 maxerr=0.00E+00 time=0.001

A sample from the optimized MPS looks like:
[2, 2]
M1
ITensor ord=2
Dim 1: (dim=2|id=198|"Link,l=1")
Dim 2: (dim=2|id=984|"S=1/2,Site,n=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 -5.904896892156415e-17  -0.9946033323558287
  0.10375071694538304    -6.159614251429185e-18
M2
ITen

(-0.7500000000000001, -0.7500000000000002, MPS
[1] ((dim=2|id=705|"Link,l=1"), (dim=2|id=29|"S=1/2,Site,n=1"))
[2] ((dim=2|id=478|"Link,l=2"), (dim=2|id=221|"S=1/2,Site,n=2"), (dim=2|id=705|"Link,l=1"))
[3] ((dim=2|id=365|"S=1/2,Site,n=3"), (dim=2|id=920|"Link,l=3"), (dim=2|id=478|"Link,l=2"))
[4] ((dim=2|id=624|"S=1/2,Site,n=4"), (dim=2|id=920|"Link,l=3"))
)

These MPS are returned in left-canonical form by the DMRG function, but we need them to be in center canonical form.

In [133]:
N0 = 2
half0 = Int(N0/2)
psi0 = orthogonalize!(psi0,half0)
U0,lambda0,V0 = svd(psi0[half0],siteind(psi0,half0));
# println("----New----")
# println(U0,lambda0,V0*psi0[2])
# println("----Old-----")
# println(psi0[1],psi0[2])
# println("----Compare Old & New----")
# println(U0*lambda0*V0*psi0[2])
# println(psi0[1]*psi0[2])

In [134]:
N1 = 4
half1 = Int(N1/2)
psi1 = orthogonalize!(psi1,half1)
combiner_ = combiner(siteind(psi1,half1),linkind(psi1,half1-1); tags="site, left link")
combined_ = combinedind(combiner_)
U1,lambda1,V1 = svd(psi1[half1]*combiner_,combined_)
U1 = dag(combiner_) * U1;
# println("----New----")
# println(psi1[1],U1,lambda1,V1*psi1[3],psi1[4])
# println("----Old-----")
# println(psi1[1],psi1[2],psi1[3],psi1[4])
# println("----Compare Old & New----")
# println(psi1[1]*U1*lambda1*V1*psi1[3]*psi1[4])
# println(psi1[1]*psi1[2]*psi1[3]*psi1[4])

Now these matrices are in the correct form:
- $\vert \Psi_0\rangle = A_0 \Lambda_0 B_0$
- $\vert \Psi_1\rangle = A_0 A_1 \Lambda_1 B_1 B_0$

Step 2 & 3: Rotate the center matrix of $\vert \Psi_1\rangle$ one step to the left and rotate the same center matrix one step to the right.

Step 2 (rotate to the left):

In [135]:
j_ = 1
psi1 = orthogonalize!(psi1,j_)
UL,lambdaL,VL = svd(psi1[j_],siteind(psi1,j_))
M1 = UL
B2 = VL*psi1[2]
B1 = psi1[3]
B0 = psi1[4]
println("Check full contraction:")
println(M1*lambdaL*B2*B1*B0)

Check full contraction:
ITensor ord=4
Dim 1: (dim=2|id=29|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=221|"S=1/2,Site,n=2")
Dim 3: (dim=2|id=365|"S=1/2,Site,n=3")
Dim 4: (dim=2|id=624|"S=1/2,Site,n=4")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×2×2
[:, :, 1, 1] =
 -0.8802264104363273     0.0
 -4.133043897886944e-16  0.0

[:, :, 2, 1] =
 3.541215771563765e-17    1.3521462393322224e-31
 1.6627540439859757e-32  -2.879705273009268e-16

[:, :, 1, 2] =
 4.987299670450511e-16  0.0
 2.341753010986269e-31  0.0

[:, :, 2, 2] =
 -2.1488931643411042e-32   2.2282362230195255e-16
 -1.0089983298374826e-47  -0.47455396570925257


Step 3 (rotate to the right):

In [136]:
j_ = 4
psi1 = orthogonalize!(psi1,j_)
UR,lambdaR,VR = svd(psi1[j_],linkind(psi1,j_-1))
A0 = psi1[1]
A1 = psi1[2]
A2 = psi1[3]*UR
M4 = VR
println("Check full contraction:")
println(A0*A1*A2*lambdaR*M4)

Check full contraction:
ITensor ord=4
Dim 1: (dim=2|id=29|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=221|"S=1/2,Site,n=2")
Dim 3: (dim=2|id=365|"S=1/2,Site,n=3")
Dim 4: (dim=2|id=624|"S=1/2,Site,n=4")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×2×2
[:, :, 1, 1] =
 -0.8802264104363273     0.0
 -4.133043897886944e-16  0.0

[:, :, 2, 1] =
 3.541215771563765e-17    1.3521462393322224e-31
 1.6627540439859757e-32  -2.879705273009268e-16

[:, :, 1, 2] =
 1.552527328398365e-16  0.0
 7.289787632887372e-32  0.0

[:, :, 2, 2] =
 -6.245931951056521e-33    2.2282362230195255e-16
 -2.9327353315989375e-48  -0.47455396570925257


Step 4: Stitch together new MPS which has been grown by two sites.

In [137]:
println("---As---")
println(A0)
println(A1)
println(A2)
println("---Lambdas---")
println(lambdaR)
println(lambda0)
println(lambdaL)
println("---Bs---")
println(B2)
println(B1)
println(B0)

---As---
ITensor ord=2
Dim 1: (dim=2|id=29|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=434|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 -1.0                    -4.695432730583713e-16
 -4.695432730583713e-16   1.0
ITensor ord=3
Dim 1: (dim=2|id=221|"S=1/2,Site,n=2")
Dim 2: (dim=2|id=434|"Link,l=1")
Dim 3: (dim=2|id=450|"Link,l=2")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×2
[:, :, 1] =
 1.0  0.0
 0.0  0.0

[:, :, 2] =
 0.0   0.0
 0.0  -1.0
ITensor ord=3
Dim 1: (dim=2|id=365|"S=1/2,Site,n=3")
Dim 2: (dim=2|id=450|"Link,l=2")
Dim 3: (dim=2|id=104|"Link,u")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×2
[:, :, 1] =
  1.0                    0.0
 -4.023073756453624e-17  3.2715506361389465e-16

[:, :, 2] =
  3.2715506361389465e-16   0.0
 -1.3161689507159756e-32  -1.0
---Lambdas---
ITensor ord=2
Dim 1: (dim=2|id=104|"Link,u")
Dim 2: (dim=2|id=186|"Link,v")
NDTensors.Diag{Float64, Vector{Float64}}
 2×2
 0.8802264104363273  0.0
 0.0                 0.47455396570925257
ITensor ord=2
